In [3]:
data_file = r'C:\Users\matematik\Documents\lab\rbcarticles.json'

In [248]:
import json
import pandas as pd
import re
from datetime import datetime

# Перетворення у DataFrame
df = pd.DataFrame(data)

# Маппінг для заміни українських назв місяців і днів тижня на англійські
months = {
    'січня': 'January', 'лютого': 'February', 'березня': 'March',
    'квітня': 'April', 'травня': 'May', 'червня': 'June',
    'липня': 'July', 'серпня': 'August', 'вересня': 'September',
    'жовтня': 'October', 'листопада': 'November', 'грудня': 'December'
}

days = {
    'Понеділок': 'Monday', 'Вівторок': 'Tuesday', 'Середа': 'Wednesday',
    'Четвер': 'Thursday', 'П\'ятниця': 'Friday', 'Субота': 'Saturday', 'Неділя': 'Sunday'
}

# Список рядків, які потрібно видалити
text_to_remove = [' | РБК-Україна', " Раніше повідомлялось, що", "Нагадаємо, ми писали."]

# Список фраз, які потрібно видаляти з кінця тексту
end_phrases_to_remove = [
    'До речі, раніше ми писали, що',
    'Нагадаємо, раніше.',
    "Раніше повідомлялося,",
    "Раніше ми писали,",
    "А ще розповідали,",
    'Також ми писали,',
    "Також ми розповідали,"
    'Ми писали,',
    " Тим часом за результатами опитування стало відомо,",
    " Нагадаємо, ми писали, що.",
    " Нагадаємо, ми писали,",
    " Нагадаємо, ми писали.",
    " Термінові та важливі повідомлення про війну Росії проти України читайте на каналі РБК-Україна в.",
    "Термінові та важливі повідомлення про війну Росії проти України читайте на каналі РБК-Україна в."
    "Термінові та важливі повідомлення про війну Росії проти України читайте на каналі.",
    "Нагадаємо, раніше ми пояснювали,",
    "Також ми розповідали, і чому це важливо.",
    "Ми писали, Також повідомлялося.",
    "Раніше ми розповідали, чи зараховується Читайте також, які",
    "Ми писали, Також за результатами свіжого опитування стало відомо,",
    "До речі, раніше ми писали, А також розповідали, що",
    "Раніше ми повідомляли, що.",
    ", - читайте в",
    "Нагадаємо, у Також повідомлялося, що.",
    "Ми писали, Також повідомлялося, як переселенцям отримати",
    "Ми розповідали, Також повідомлялося.",
    "Також ми розповідали, що",
    "Докладніше про нові тарифи на ці напрямки –.",
    "Також ми розповідали.",
    "Нагадаємо, Також українка поділилася з РБК-Україна своєю думкою про те,",
    "РБК-Україна також писало, що.",
    "Термінові та важливі повідомлення про війну Росії проти України читайте на каналі.",
    "Нагадаємо, ми розповідали."

]

# Функція для заміни українських назв на англійські
def replace_uk_to_en(date_str):
    for ua_month, en_month in months.items():
        date_str = date_str.replace(ua_month, en_month)
    for ua_day, en_day in days.items():
        date_str = date_str.replace(ua_day, en_day)
    return date_str

# Заміна назв місяців і днів та перетворення у datetime
df['article_datetime'] = df['article_datetime'].apply(replace_uk_to_en)
df['article_datetime'] = pd.to_datetime(df['article_datetime'], format='%A %d %B %Y %H:%M ')

# Функція для очищення тексту від технічних символів і видалення рядків
def clean_text(text):
    # Видалення екрануючих символів, зайвих пробілів, нових рядків і табуляцій
    text = re.sub(r'\\[tn]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()


    # Видалення фраз
    for phrase in end_phrases_to_remove:
        text = text.replace(phrase, '')

    # Видалення вказаних рядків
    for t in text_to_remove:
        text = text.replace(t, '')
    # Видалення пробілів перед крапкою
    text = re.sub(r'\s+\.', '.', text)
    # Видалення пробілів перед комами
    text = re.sub(r'\s+,', ',', text)
    # Видалення повторюваних ком
    text = re.sub(r',+', ',', text)
    # Додавання пробілу після коми, якщо його немає
    text = re.sub(r',(\S)', r', \1', text)
    # Перетворення екранованих апострофів на звичайні
    text = re.sub(r"\\'", "'", text)
    # Заміна ", ." на крапку
    text = re.sub(r',\s*\.', '.', text)

    text = re.sub(rf'{re.escape("Зокрема.")}$', '.', text)
    text = re.sub(rf'{re.escape(" Також минулого тижня у Німеччині.")}$', '.', text)
    text = re.sub(rf'{re.escape("Термінові та важливі повідомлення про війну Росії проти України читайте на каналі.")}$', '.', text)
    text = re.sub(rf'{re.escape("Також ми повідомляли,")}$', '.', text)
    text = re.sub(rf'{re.escape(", де.")}$', '.', text)
    text = re.sub(rf'{re.escape("Нагадаємо, ми повідомляли, що.")}$', '.', text)
    text = re.sub(rf'{re.escape("чому .")}$', '.', text)
    text = re.sub(rf'{re.escape("Ми писали, ")}$', '.', text)
    text = re.sub(rf'{re.escape("Також ми розповідали,")}$', '.', text)
    text = re.sub(rf'{re.escape("  що Крім того,")}$', '.', text)
    text = re.sub(rf'{re.escape(".  що")}$', '.', text)
    text = re.sub(rf'{re.escape(".  що.")}$', '.', text)
    text = re.sub(rf'{re.escape(". що")}$', '.', text)
    text = re.sub(rf'{re.escape(". що.")}$', '.', text)
    text = re.sub(rf'{re.escape(",")}$', '.', text)
    # Видалення повторюваних ком та крапок після очищення
    text = re.sub(r'\.\s\.', ',', text)
    text = re.sub(r',+', ',', text)
    text = re.sub(r'\.\.+', '.', text)

    return text

# Очистка тексту
df['article_title'] = df['article_title'].apply(clean_text)
df['article_text'] = df['article_text'].apply(clean_text)

df.sample(3).transpose()

,688,222,2192
article_url,https://www.rbc.ua/rus/styler/tse-ilyuziya-biz...,https://www.rbc.ua/rus/news/timchasoviy-zahist...,https://www.rbc.ua/rus/news/rivnenskiy-oblasti...
article_uuid,0356f62afa0e06b0b922116dde5fe88935fa8e7307af17...,3ee128358d373e6b691dac5db1f1c2c2cc3b84a8474f3f...,8bec6f677ec10b083fc0b62c13e2280bd96e08a0a3d333...
article_id,1698922428,1705654549,1666960772
article_datetime,2023-11-02 14:10:00,2024-01-19 11:00:00,2022-10-28 15:39:00
article_title,Виїзд біженців за кордон - українка пояснила ч...,Українські біженці у Польщі – тимчасовий захис...,Переселенців у Рівненській області перевірять ...
article_text,"Українка, яка проживає з дітьми в Англії, розп...",Про це повідомляє з на Посольство України у По...,У Рівненській області перевірять внутрішньопер...


In [249]:
current_index = 12

def print_next_article(start_from=None):
    global current_index
    if start_from is not None:
        current_index = start_from

    if current_index < len(df):
        article_text = df.iloc[current_index].article_text
        print('Стаття №', current_index + 1)
        print(article_text)
        current_index += 1
    else:
        print("Більше статей немає.")


In [257]:
print_next_article()

Стаття № 16
Які номерні знаки можуть отримати біженці на придбане у Чехії авто, розповідає з посиланням на. Українці, які Чехії перебувають із тимчасовим захистом та візами толерантності, можуть придбати автомобіль в цій країні чи в іншій країні ЄС. Але не можуть отримати стандартний чеський номерний знак, пояснили виданню у Міністерстві транспорту Чехії. У відомстві зазначили, що власником транспортного засобу, зареєстрованого в Реєстрі дорожніх транспортних засобів, може бути іноземець з тимчасовим захистом або візою толерантності. "Але водієм транспортного засобу відповідно до закону про умови експлуатації транспортних засобів на дорогах, може бути особа, яка має дозвіл на постійне проживання (trvalý pobyt), дозвіл на довгострокове проживання (dlouhodobý pobyt) або яка отримала притулок (azyl) на території Чеської Республіки", – пояснили у Мінтрансі. Це означає, що тимчасовий захист не входить до категорії дозволів на перебування, що дозволяють чеську реєстрацію авто. Українці із ти